In [0]:
%run ../initializeJobVariables

In [0]:
from pyspark.sql import functions as F
import uuid
run_token = str(uuid.uuid4())
environment = "development"
dict_job_data = getRunDetails()
pipeline_name = dict_job_data["jobName"]
job_id = dict_job_data["jobId"]
job_run_id = dict_job_data["jobRunId"]
pipeline_status = 1
df = (spark.table(f"{environment}_011_bronze_core.db_admin.pipeline")
          .filter(F.col("pipeline_name") == pipeline_name))

pipeline_row = df.select("pipeline_id").first()
pipeline_id = pipeline_row["pipeline_id"]
sql = f"""
INSERT INTO {environment}_011_bronze_core.db_admin.pipeline_history
(pipeline_id,job_id, job_run_id, start_time, pipeline_status, run_token)
VALUES (
try_cast({pipeline_id} as bigint),
{job_id}, 
{job_run_id},
current_timestamp(),
{pipeline_status},
'{run_token}'
)
"""
spark.sql(sql)